# Group 1 Final Project Work 

#### Specific data on our dataset

### Maternal Health Risk Dataset Summary

**Shape:** 808 records × 7 columns  

**Columns:**
- `Age`
- `SystolicBP` (Systolic Blood Pressure)
- `DiastolicBP` (Diastolic Blood Pressure)
- `BS` (Blood Sugar level)
- `BodyTemp` (Body Temperature, °F)
- `HeartRate` (Heart Rate, bpm)
- `RiskLevel` (Target: maternal health risk category)

---

#### First 5 Records
| Age | SystolicBP | DiastolicBP | BS   | BodyTemp | HeartRate | RiskLevel  |
|-----|------------|--------------|------|----------|-----------|------------|
| 25  | 130        | 80           | 15.0 | 98.0     | 86        | high risk  |
| 35  | 140        | 90           | 13.0 | 98.0     | 70        | high risk  |
| 29  | 90         | 70           | 8.0  | 100.0    | 80        | high risk  |
| 30  | 140        | 85           | 7.0  | 98.0     | 70        | high risk  |
| 35  | 120        | 60           | 6.1  | 98.0     | 76        | low risk   |

---

#### Summary Statistics
- **Age:** 10–70 years (mean = 30.6, std = 13.9)  
- **SystolicBP:** 70–160 mmHg (mean = 113, std = 19.9)  
- **DiastolicBP:** 49–100 mmHg (mean = 77.5, std = 14.8)  
- **BS:** 6–19 mmol/L (mean = 9.26, std = 3.62)  
- **BodyTemp:** 98–103 °F (mean = 98.6, std = 1.39)  
- **HeartRate:** 7–90 bpm (mean = 74.3, std = 8.82)  

---

#### Target Variable: RiskLevel
- **Low risk:** 478 records (~59.2%)  
- **High risk:** 330 records (~40.8%)  
- **Medium risk:** Not present in this dataset version  

 Note: The dataset is binary-labeled (low vs. high risk), so if a 3-class model (low/mid/high) is needed, additional data preprocessing or augmentation may be required.

### Week 3 - Training and Feature Engineering

#### Environment (auto role + auto bucket) and constants

In [1]:
# NO MANUAL SETTINGS: bucket/role are auto-detected from your Studio kernel.

import os, io, json, time
from pathlib import Path
import boto3, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

plt.rcParams["figure.dpi"] = 120

# AWS context (auto from Studio kernel)
boto_sess  = boto3.session.Session()
region     = boto_sess.region_name
sm_session = Session(boto_sess)
role       = get_execution_role()
bucket     = sm_session.default_bucket()

RUN_ID    = time.strftime("%Y%m%d-%H%M%S")
S3_PREFIX = f"aai540/maternal-risk/week3/{RUN_ID}"

# Paths
DATA_CSV      = Path("Maternal_Risk.csv")    # Using our Kaggle CSV here
ARTIFACTS_DIR = Path("week3_outputs"); ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

print("Region:", region)
print("Role:  ", role)
print("S3:    ", f"s3://{bucket}/{S3_PREFIX}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Region: us-east-1
Role:   arn:aws:iam::849121223812:role/LabRole
S3:     s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711


#### Load data + lightweight EDA (plots + JSON summary)

In [2]:
assert DATA_CSV.exists(), f"Missing dataset at {DATA_CSV.resolve()}"
df = pd.read_csv(DATA_CSV)

# Keep a small EDA summary to feed the team tracker
eda_summary = {
    "rows": int(df.shape[0]),
    "cols": int(df.shape[1]),
    "columns": df.columns.tolist(),
    "dtypes": {c: str(t) for c, t in df.dtypes.items()},
    "missing_counts": df.isna().sum().to_dict(),
    "class_counts": df["RiskLevel"].value_counts().to_dict(),
}
json.dump(eda_summary, open(ARTIFACTS_DIR/"eda_summary.json","w"), indent=2)

# A few simple plots for the design doc
(df["RiskLevel"].value_counts()
   .plot(kind="bar", title="Class Distribution")
   .get_figure().savefig(ARTIFACTS_DIR/"chart_class_distribution.png")); plt.clf()

df["Age"].plot(kind="hist", bins=20, title="Age Distribution").get_figure().savefig(
    ARTIFACTS_DIR/"chart_age_hist.png"); plt.clf()

plt.boxplot([df["SystolicBP"], df["DiastolicBP"]], tick_labels=["SystolicBP","DiastolicBP"])
plt.title("Blood Pressure Boxplots"); plt.tight_layout()
plt.savefig(ARTIFACTS_DIR/"chart_bp_box.png"); plt.clf()

num_cols = df.select_dtypes(include=[np.number]).columns
corr = df[num_cols].corr()
plt.imshow(corr, interpolation="nearest"); plt.colorbar()
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
plt.yticks(range(len(num_cols)), num_cols)
plt.title("Correlation Heatmap"); plt.tight_layout()
plt.savefig(ARTIFACTS_DIR/"chart_corr_heatmap.png"); plt.clf()

print("EDA done -->", ARTIFACTS_DIR)

EDA done --> week3_outputs


<Figure size 768x576 with 0 Axes>

#### Feature engineering (clinically-motivated features + z-scaling)

In [3]:
# We derive simple vitals-based features and also add z-scaled versions for linear models.
X = df.copy()
# simple clinically meaningful features
X["PulsePressure"]    = X["SystolicBP"] - X["DiastolicBP"]
X["SBP_to_DBP"]       = X["SystolicBP"] / (X["DiastolicBP"].replace(0, np.nan))
X["Fever"]            = (X["BodyTemp"] > 99.5).astype(int)
X["Tachycardia"]      = (X["HeartRate"] >= 100).astype(int)
X["HypertensionFlag"] = ((X["SystolicBP"] >= 140) | (X["DiastolicBP"] >= 90)).astype(int)

# z-scaling for linear models
cont = ["Age","SystolicBP","DiastolicBP","BS","BodyTemp","HeartRate","PulsePressure"]
X[[f"z_{c}" for c in cont]] = StandardScaler().fit_transform(X[cont])

# binary labels (in this dataset: "low risk" / "high risk")
label_map = {"low risk": 0, "high risk": 1}
y = X["RiskLevel"].map(label_map)
engineered = pd.concat([X.drop(columns=["RiskLevel"]), y.rename("label")], axis=1)
engineered.to_csv(ARTIFACTS_DIR/"maternal_features_full.csv", index=False)
json.dump(label_map, open(ARTIFACTS_DIR/"label_map.json","w"), indent=2)

# Stratified splits (train/val/test/prod = 40/10/10/40)
X_no_target = engineered.drop(columns=["label"])
y_only      = engineered["label"]

X_tmp, X_prod, y_tmp, y_prod = train_test_split(
    X_no_target, y_only, test_size=0.40, random_state=42, stratify=y_only
)
X_train, X_rem, y_train, y_rem = train_test_split(
    X_tmp, y_tmp, test_size=(1/3), random_state=42, stratify=y_tmp
)
X_val, X_test, y_val, y_test = train_test_split(
    X_rem, y_rem, test_size=0.5, random_state=42, stratify=y_rem
)

def save_split(name, Xd, yd):
    out = Xd.copy(); out["label"] = yd.values
    out.to_csv(ARTIFACTS_DIR/f"{name}.csv", index=False)
    return out

train_df = save_split("train", X_train, y_train)
val_df   = save_split("val",   X_val,   y_val)
test_df  = save_split("test",  X_test,  y_test)
prod_df  = save_split("production", X_prod, y_prod)

print("Feature engineering done.")

Feature engineering done.


#### Stratified splits: 40% prod, 40% train, 10% val, 10% test

In [4]:
# We first carve out 40% as "production" holdout for future batch inference/monitoring.
# The remaining 60% --> train (40%), val (10%), test (10%) of the original dataset.

from sklearn.model_selection import train_test_split

# 40% set aside for future batch inference/monitoring
X_tmp, X_prod, y_tmp, y_prod = train_test_split(
    X_no_target, y, test_size=0.40, random_state=42, stratify=y
)
# remaining 60% -> 40/10/10
X_train, X_rem, y_train, y_rem = train_test_split(
    X_tmp, y_tmp, test_size=(1/3), random_state=42, stratify=y_tmp
)
X_val, X_test, y_val, y_test = train_test_split(
    X_rem, y_rem, test_size=0.5, random_state=42, stratify=y_rem
)

def _save(name, Xd, yd):
    out = Xd.copy(); out["label"] = yd.values
    out.to_csv(ARTIFACTS_DIR / f"{name}.csv", index=False)
    return out

train_df = _save("train",      X_train, y_train)
val_df   = _save("val",        X_val,   y_val)
test_df  = _save("test",       X_test,  y_test)
prod_df  = _save("production", X_prod,  y_prod)

print({"train":len(train_df), "val":len(val_df), "test":len(test_df), "production":len(prod_df)})

{'train': 322, 'val': 81, 'test': 81, 'production': 324}


#### Upload artifacts to S3 (no manual bucket)

In [5]:
# Upload the CSVs, label map, EDA summary, and figures to your default bucket/prefix.

s3 = boto3.client("s3")

def s3_upload(local: Path, key: str):
    s3.upload_file(str(local), bucket, f"{S3_PREFIX}/{key}")
    print("Uploaded", f"s3://{bucket}/{S3_PREFIX}/{key}")

# CSVs + summaries
for fname in ["train.csv","val.csv","test.csv","production.csv",
              "maternal_features_full.csv","label_map.json","eda_summary.json"]:
    s3_upload(ARTIFACTS_DIR / fname, fname)

# Plots
for fname in ["chart_class_distribution.png","chart_age_hist.png","chart_bp_box.png","chart_corr_heatmap.png"]:
    s3_upload(ARTIFACTS_DIR / fname, f"figures/{fname}")

Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/train.csv
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/val.csv
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/test.csv
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/production.csv
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/maternal_features_full.csv
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/label_map.json
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/eda_summary.json
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/figures/chart_class_distribution.png
Uploaded s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711/figures/chart_age_hist.png
Uploaded s3://sagemaker-u

#### Sanitize column names (Feature Store regex) & write sanitized splits

In [6]:
# FS rules: names must be letters/numbers/hyphens only; must start with alnum; <=64 chars.

def sanitize_col(name: str) -> str:
    if name == "SBP_to_DBP": name = "SBPtoDBP"   # preserve meaning
    if name.startswith("z_"): name = "z" + name[2:]
    name = name.replace("_", "")
    name = "".join(ch for ch in name if ch.isalnum() or ch == "-")
    if not name or not name[0].isalnum(): name = "f" + name
    return name[:64]

def sanitize_df_cols(df: pd.DataFrame) -> pd.DataFrame:
    newcols, seen = [], set()
    for c in df.columns:
        s = sanitize_col(c)
        if s in seen:
            i, base = 2, s
            while f"{base}{i}" in seen: i += 1
            s = f"{base}{i}"
        newcols.append(s); seen.add(s)
    out = df.copy(); out.columns = newcols
    return out

label_col = "label"
def sanitize_split(df):
    feats = df.drop(columns=[label_col])
    feats = sanitize_df_cols(feats)
    feats[label_col] = df[label_col].values
    return feats

train_s = sanitize_split(train_df); train_s.to_csv(ARTIFACTS_DIR/"train_sanitized.csv", index=False)
val_s   = sanitize_split(val_df);   val_s.to_csv(ARTIFACTS_DIR/"val_sanitized.csv",   index=False)
test_s  = sanitize_split(test_df);  test_s.to_csv(ARTIFACTS_DIR/"test_sanitized.csv", index=False)
prod_s  = sanitize_split(prod_df);  prod_s.to_csv(ARTIFACTS_DIR/"production_sanitized.csv", index=False)

print("Sanitized splits saved.")

Sanitized splits saved.


#### Create & ingest Feature Store (OFFLINE, unique names per run)

In [7]:
import time
import boto3
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import FeatureDefinition, FeatureTypeEnum

sm      = boto3.client("sagemaker")
session = Session(boto3.session.Session(region_name=region))

def ensure_id_time(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    if "recordid" not in df.columns:
        df["recordid"] = range(1, len(df)+1)
    if "eventtime" not in df.columns:
        df["eventtime"] = pd.Timestamp.utcnow().isoformat()
    return df

def to_boto_feature_defs(df: pd.DataFrame):
    out = []
    for c, d in df.dtypes.items():
        if c == "eventtime":
            t = "String"
        elif pd.api.types.is_integer_dtype(d):
            t = "Integral"
        elif pd.api.types.is_float_dtype(d):
            t = "Fractional"
        else:
            t = "String"
        out.append({"FeatureName": c, "FeatureType": t})
    return out

def create_fg_boto3(name: str, df_local: pd.DataFrame, s3_uri: str):
    fdefs = to_boto_feature_defs(df_local)
    try:
        resp = sm.create_feature_group(
            FeatureGroupName=name,
            RecordIdentifierFeatureName="recordid",
            EventTimeFeatureName="eventtime",
            FeatureDefinitions=fdefs,
            OfflineStoreConfig={"S3StorageConfig": {"S3Uri": s3_uri}},
            OnlineStoreConfig={"EnableOnlineStore": False},
            RoleArn=role,
            Description=f"Maternal Health Risk – {name}",
        )
        return resp
    except sm.exceptions.ResourceInUse:
        # Already exists --> safe to reuse after we confirm it's Created
        return {"FeatureGroupArn": f"arn:aws:sagemaker:{region}:{boto3.client('sts').get_caller_identity()['Account']}:feature-group/{name}"}

def wait_fg_created(name: str, timeout_s: int = 900, poll_s: int = 10):
    start = time.time()
    last = ""
    while True:
        desc = sm.describe_feature_group(FeatureGroupName=name)
        status = desc.get("FeatureGroupStatus", "")
        if status == "Created":
            print(f"[READY] {name}")
            return desc
        if status == "CreateFailed":
            raise RuntimeError(f"{name} failed: {desc.get('FailureReason')}")
        if time.time() - start > timeout_s:
            raise TimeoutError(f"Timeout waiting for {name} (last status={status})")
        if status != last:
            print(f"Status {name}: {status}")
            last = status
        time.sleep(poll_s)

def create_and_ingest(name_base: str, df_local: pd.DataFrame):
    # unique FG names per run to avoid collisions
    name = f"{name_base}-{RUN_ID}"             # e.g., mhr-train-fg-20250920-154301
    assert "_" not in name, "FG name must not contain underscores."
    df_local = ensure_id_time(df_local)
    s3_uri   = f"s3://{bucket}/{S3_PREFIX}/feature-store/{name}"

    create_fg_boto3(name, df_local, s3_uri)
    wait_fg_created(name)

    fg = FeatureGroup(name=name, sagemaker_session=session)
    fg.load_feature_definitions(data_frame=df_local)    # make sure SDK knows schema
    fg.ingest(data_frame=df_local, max_workers=4, wait=True)
    print(f"[OK] Ingested {len(df_local)} rows → {name}")
    return name

# Load sanitized splits
train_s = pd.read_csv(ARTIFACTS_DIR/"train_sanitized.csv")
val_s   = pd.read_csv(ARTIFACTS_DIR/"val_sanitized.csv")
prod_s  = pd.read_csv(ARTIFACTS_DIR/"production_sanitized.csv")

# Create OFFLINE FGs with unique names
FG_TRAIN = create_and_ingest("mhr-train-fg", train_s)
FG_VAL   = create_and_ingest("mhr-val-fg",   val_s)
FG_BATCH = create_and_ingest("mhr-batch-fg", prod_s)

print("Feature Store complete:", FG_TRAIN, FG_VAL, FG_BATCH)

Status mhr-train-fg-20251007-221711: Creating
[READY] mhr-train-fg-20251007-221711
[OK] Ingested 322 rows → mhr-train-fg-20251007-221711
Status mhr-val-fg-20251007-221711: Creating
[READY] mhr-val-fg-20251007-221711
[OK] Ingested 81 rows → mhr-val-fg-20251007-221711
Status mhr-batch-fg-20251007-221711: Creating
[READY] mhr-batch-fg-20251007-221711
[OK] Ingested 324 rows → mhr-batch-fg-20251007-221711
Feature Store complete: mhr-train-fg-20251007-221711 mhr-val-fg-20251007-221711 mhr-batch-fg-20251007-221711


#### Tracker update (JSON + Markdown) and upload

In [8]:
tracker = {
    "run_id": RUN_ID,
    "s3_prefix": f"s3://{bucket}/{S3_PREFIX}",
    "dataset": {
        "rows": eda_summary["rows"], "cols": eda_summary["cols"],
        "class_counts": eda_summary["class_counts"],
        "dtypes": eda_summary["dtypes"],
        "missing": eda_summary["missing_counts"],
    },
    "splits": {
        "train_rows": len(train_df), "val_rows": len(val_df),
        "test_rows": len(test_df), "prod_rows": len(prod_df),
    },
    "feature_store_groups": [FG_TRAIN, FG_VAL, FG_BATCH],
}
with open(ARTIFACTS_DIR / "team_tracker_update_week3.json", "w") as f:
    json.dump(tracker, f, indent=2)

md = f"""# Week 3 Tracker — Maternal Health Risk (RUN: {RUN_ID})

**S3 prefix:** s3://{bucket}/{S3_PREFIX}

## Dataset
- Rows: {eda_summary['rows']} | Cols: {eda_summary['cols']}
- Classes: {eda_summary['class_counts']}

## Splits
- Train: {len(train_df)} (~40%)
- Val:   {len(val_df)} (~10%)
- Test:  {len(test_df)} (~10%)
- Prod:  {len(prod_df)} (~40%)

## Feature Store (offline)
- {FG_TRAIN}
- {FG_VAL}
- {FG_BATCH}
"""
with open(ARTIFACTS_DIR / "team_tracker_update_week3.md", "w") as f:
    f.write(md)

# Upload tracker docs
s3 = boto3.client("s3")
s3.upload_file(str(ARTIFACTS_DIR/"team_tracker_update_week3.json"), bucket, f"{S3_PREFIX}/team_tracker_update_week3.json")
s3.upload_file(str(ARTIFACTS_DIR/"team_tracker_update_week3.md"),   bucket, f"{S3_PREFIX}/team_tracker_update_week3.md")

print("Tracker written & uploaded.")

Tracker written & uploaded.


In [9]:
# View

import boto3, json
s3 = boto3.client("s3")
obj = s3.get_object(Bucket=bucket, Key=f"{S3_PREFIX}/team_tracker_update_week3.json")
tracker = json.load(obj["Body"])
tracker

{'run_id': '20251007-221711',
 's3_prefix': 's3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711',
 'dataset': {'rows': 808,
  'cols': 7,
  'class_counts': {'low risk': 478, 'high risk': 330},
  'dtypes': {'Age': 'int64',
   'SystolicBP': 'int64',
   'DiastolicBP': 'int64',
   'BS': 'float64',
   'BodyTemp': 'float64',
   'HeartRate': 'int64',
   'RiskLevel': 'object'},
  'missing': {'Age': 0,
   'SystolicBP': 0,
   'DiastolicBP': 0,
   'BS': 0,
   'BodyTemp': 0,
   'HeartRate': 0,
   'RiskLevel': 0}},
 'splits': {'train_rows': 322,
  'val_rows': 81,
  'test_rows': 81,
  'prod_rows': 324},
 'feature_store_groups': ['mhr-train-fg-20251007-221711',
  'mhr-val-fg-20251007-221711',
  'mhr-batch-fg-20251007-221711']}

## Week 4, Model Development and Deployment

#### Auto settings; continues from Week 3

In [10]:
"""
AAI-540 — Week 4: Benchmark (LogReg), Main Model (XGBoost), Evaluation, Batch Deploy
This script:
  1) Reuses Week-3 splits from the latest run (or pass WEEK3_PREFIX env var)
  2) Trains a *benchmark* Logistic Regression (Age + SystolicBP) via SKLearn Estimator
  3) Trains a full-feature **XGBoost** model (built-in container)
  4) Compares metrics on the test set and writes confusion matrices
  5) Runs **Batch Transform** on Week-3 production data
  6) Writes a Week-4 tracker + design-doc snippet with S3 artifact links
"""

import os, io, json, time, tarfile
from pathlib import Path
import boto3, sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
import pandas as pd
import numpy as np

# Reuse Week-3 objects if they exist; otherwise, auto-init (no manual config)
try:
    bucket
    sm_session
    role
except NameError:
    boto_sess  = boto3.session.Session()
    sm_session = Session(boto_sess)
    role       = get_execution_role()
    bucket     = sm_session.default_bucket()

# Use the Week-3 S3 prefix if it’s still in memory; otherwise pick the latest run
s3 = boto3.client("s3")
try:
    WEEK3_PREFIX = S3_PREFIX  # from Week 3 cells
except NameError:
    base = "aai540/maternal-risk/week3/"
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=base, Delimiter="/")
    runs = [cp["Prefix"].rstrip("/") for cp in resp.get("CommonPrefixes", [])]
    assert runs, f"No Week-3 artifacts found under s3://{bucket}/{base}"
    WEEK3_PREFIX = sorted(runs)[-1]

# Create a unique Week-4 prefix
RUN_ID = time.strftime("%Y%m%d-%H%M%S")
W4_PREFIX = f"aai540/maternal-risk/week4/{RUN_ID}"

print("Using Week-3:", f"s3://{bucket}/{WEEK3_PREFIX}")
print("Writing Week-4:", f"s3://{bucket}/{W4_PREFIX}")

Using Week-3: s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711
Writing Week-4: s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946


#### Load Week-3 splits (train/val/test) from S3

In [11]:
# LOAD SPLITS

def read_csv_from_s3(key: str) -> pd.DataFrame:
    obj = s3.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(io.BytesIO(obj["Body"].read()))

train = read_csv_from_s3(f"{WEEK3_PREFIX}/train.csv")
val   = read_csv_from_s3(f"{WEEK3_PREFIX}/val.csv")
test  = read_csv_from_s3(f"{WEEK3_PREFIX}/test.csv")

label_col = "label"
X_train, y_train = train.drop(columns=[label_col]), train[label_col]
X_val,   y_val   = val.drop(columns=[label_col]),   val[label_col]
X_test,  y_test  = test.drop(columns=[label_col]),  test[label_col]

print("Loaded:", train.shape, val.shape, test.shape)
print("Train label balance:", y_train.value_counts().to_dict())

Loaded: (322, 19) (81, 19) (81, 19)
Train label balance: {0: 190, 1: 132}


#### Benchmark model in SageMaker (very simple: Logistic Regression on 2 features)

In [12]:
# Why: have a simple, interpretable baseline for comparison (MVP).
from sagemaker.sklearn import SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

bm_feats = ["Age", "SystolicBP"]
assert all(f in X_train.columns for f in bm_feats), "Expected baseline features missing."

# Stage small CSVs (filenames don't matter inside channels)
w4_local = Path("w4_benchmark"); w4_local.mkdir(exist_ok=True)
pd.concat([X_train[bm_feats], y_train], axis=1).to_csv(w4_local/"train_benchmark.csv", index=False)
pd.concat([X_val[bm_feats],   y_val],   axis=1).to_csv(w4_local/"val_benchmark.csv",   index=False)

bm_train_s3 = sm_session.upload_data(str(w4_local/"train_benchmark.csv"), key_prefix=f"{W4_PREFIX}/benchmark")
bm_val_s3   = sm_session.upload_data(str(w4_local/"val_benchmark.csv"),   key_prefix=f"{W4_PREFIX}/benchmark")

# Entry script: read first *.csv in each channel, fit LR, write metrics + model
with open("baseline_train.py","w") as f:
    f.write("""
import os, glob, json, pathlib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

def first_csv_in(d):
    files = sorted(glob.glob(os.path.join(d, '*.csv')))
    assert files, f'No CSV found in {d}'
    return files[0]

if __name__ == '__main__':
    train_dir = os.environ.get('SM_CHANNEL_TRAIN', '/opt/ml/input/data/train')
    val_dir   = os.environ.get('SM_CHANNEL_VAL',   '/opt/ml/input/data/val')
    model_dir = os.environ.get('SM_MODEL_DIR',     '/opt/ml/model')

    df_tr = pd.read_csv(first_csv_in(train_dir))
    df_va = pd.read_csv(first_csv_in(val_dir))

    Xtr, ytr = df_tr[['Age','SystolicBP']], df_tr['label']
    Xva, yva = df_va[['Age','SystolicBP']], df_va['label']

    clf = LogisticRegression(max_iter=1000).fit(Xtr, ytr)

    pred  = clf.predict(Xva)
    proba = clf.predict_proba(Xva)[:,1]
    acc = accuracy_score(yva, pred)
    p,r,f1,_ = precision_recall_fscore_support(yva, pred, average='binary', zero_division=0)
    try: auc = roc_auc_score(yva, proba)
    except: auc = float('nan')

    pathlib.Path(model_dir).mkdir(parents=True, exist_ok=True)
    import joblib
    joblib.dump(clf, os.path.join(model_dir, 'model.joblib'))
    with open(os.path.join(model_dir, 'metrics.json'), 'w') as f:
        json.dump({'accuracy':acc,'precision':p,'recall':r,'f1':f1,'roc_auc':auc}, f)
""")

bm_est = SKLearn(
    entry_point="baseline_train.py",
    framework_version="1.2-1",     # use a tag compatible with your Studio image
    role=role,
    instance_type="ml.m5.large",
    instance_count=1,
    sagemaker_session=sm_session,
)
bm_est.fit({"train": bm_train_s3, "val": bm_val_s3})

# Also compute baseline metrics on our held-out TEST for a clean comparison
bm_clf   = LogisticRegression(max_iter=1000).fit(train[bm_feats], y_train)
bm_proba = bm_clf.predict_proba(test[bm_feats])[:,1]
bm_pred  = (bm_proba >= 0.5).astype(int)

bm_acc = accuracy_score(y_test, bm_pred)
bm_p, bm_r, bm_f1, _ = precision_recall_fscore_support(y_test, bm_pred, average='binary', zero_division=0)
try:    bm_auc = roc_auc_score(y_test, bm_proba)
except: bm_auc = float("nan")

baseline_metrics = {"accuracy":bm_acc,"precision":bm_p,"recall":bm_r,"f1":bm_f1,"roc_auc":bm_auc}
baseline_metrics

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2025-10-07-22-19-52-447


2025-10-07 22:19:53 Starting - Starting the training job...
2025-10-07 22:20:07 Starting - Preparing the instances for training...
2025-10-07 22:20:54 Downloading - Downloading the training image......../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-10-07 22:22:02,796 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-10-07 22:22:02,800 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-07 22:22:02,803 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-10-07 22:22:02,819 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-10-07 22:22:03,129 s

{'accuracy': 0.7654320987654321,
 'precision': 0.71875,
 'recall': 0.696969696969697,
 'f1': 0.7076923076923077,
 'roc_auc': 0.790719696969697}

#### MAIN MODEL in SageMaker (Built-in XGBoost, CSV mode)

In [13]:
# Why built-in? No entry_point needed; just CSV with label first (no header).
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
import sagemaker

# Helper to reorder columns to [label, features...] for CSV training
def reorder_for_xgb(df):
    cols = [label_col] + [c for c in df.columns if c != label_col]
    return df[cols]

xgb_train_local = reorder_for_xgb(train)
xgb_val_local   = reorder_for_xgb(val)

xgb_train_path = Path("w4_xgb_train.csv"); xgb_val_path = Path("w4_xgb_val.csv")
xgb_train_local.to_csv(xgb_train_path, index=False, header=False)
xgb_val_local.to_csv(xgb_val_path,   index=False, header=False)

s3_xgb_train = sm_session.upload_data(str(xgb_train_path), key_prefix=f"{W4_PREFIX}/xgb")
s3_xgb_val   = sm_session.upload_data(str(xgb_val_path),   key_prefix=f"{W4_PREFIX}/xgb")

def get_xgb_image():
    for ver in ["1.7-1", "1.5-1", "1.3-1"]:
        try:
            return sagemaker.image_uris.retrieve("xgboost", sm_session.boto_region_name, version=ver)
        except Exception as e:
            print(f"xgboost {ver} not available → trying next … ({e})")
    raise RuntimeError("No compatible built-in XGBoost image found.")

xgb_image_uri = get_xgb_image()

xgb_est = Estimator(
    image_uri=xgb_image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    sagemaker_session=sm_session,
    hyperparameters={
        "objective":"binary:logistic",
        "eval_metric":"auc",
        "max_depth":5,
        "eta":0.2,
        "min_child_weight":1,
        "subsample":0.8,
        "colsample_bytree":0.8,
        "num_round":200,
        "verbosity":1,
    },
)

# tell container the training data is CSV (otherwise it expects libsvm)
train_input = TrainingInput(s3_data=s3_xgb_train, content_type="text/csv")
val_input   = TrainingInput(s3_data=s3_xgb_val,   content_type="text/csv")

xgb_est.fit({"train": train_input, "validation": val_input}, wait=True)
xgb_model_artifact = xgb_est.model_data
print("XGBoost model artifact:", xgb_model_artifact)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-10-07-22-24-35-764


2025-10-07 22:24:38 Starting - Starting the training job...
2025-10-07 22:24:53 Starting - Preparing the instances for training...
2025-10-07 22:25:12 Downloading - Downloading input data...
2025-10-07 22:25:52 Downloading - Downloading the training image......
2025-10-07 22:27:09 Training - Training image download completed. Training in progress.
2025-10-07 22:27:09 Uploading - Uploading generated training model/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-10-07 22:27:01.074 ip-10-2-254-89.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-10-07 22:27:01.151 ip-10-2-254-89.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-10-07:22:

#### EVALUATE (compare main vs baseline on TEST)

In [14]:
import xgboost as xgb

def parse_s3_uri(uri: str):
    assert uri.startswith("s3://")
    p = uri[5:]; b, k = p.split("/", 1)
    return b, k

tmp_dir = Path("w4_tmp"); tmp_dir.mkdir(exist_ok=True)
bkt, key = parse_s3_uri(xgb_model_artifact)
boto3.client("s3").download_file(bkt, key, str(tmp_dir/"model.tar.gz"))
with tarfile.open(tmp_dir/"model.tar.gz") as t:
    t.extractall(tmp_dir)

# Score test set with the trained booster
dtest   = xgb.DMatrix(test.drop(columns=[label_col]), label=test[label_col])
booster = xgb.Booster(); booster.load_model(str(tmp_dir/"xgboost-model"))
xgb_proba = booster.predict(dtest)
xgb_pred  = (xgb_proba >= 0.5).astype(int)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_p, xgb_r, xgb_f1, _ = precision_recall_fscore_support(y_test, xgb_pred, average='binary', zero_division=0)
xgb_auc = roc_auc_score(y_test, xgb_proba)

metrics_compare = {
    "baseline": baseline_metrics,
    "xgboost":  {"accuracy":xgb_acc,"precision":xgb_p,"recall":xgb_r,"f1":xgb_f1,"roc_auc":xgb_auc},
}
metrics_compare

/tmp/ipykernel_228/132528516.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(tmp_dir)


{'baseline': {'accuracy': 0.7654320987654321,
  'precision': 0.71875,
  'recall': 0.696969696969697,
  'f1': 0.7076923076923077,
  'roc_auc': 0.790719696969697},
 'xgboost': {'accuracy': 0.9876543209876543,
  'precision': 1.0,
  'recall': 0.9696969696969697,
  'f1': 0.9846153846153847,
  'roc_auc': 0.999368686868687}}

#### Deploy via Batch Transform (score Week-3 production.csv)

In [15]:
# Inference expects FEATURES ONLY (no label) in the SAME order as training.

from sagemaker.inputs import TransformInput

prod_df = read_csv_from_s3(f"{WEEK3_PREFIX}/production.csv")

# Same feature order as used to create training CSVs
FEATURE_COLS = [c for c in train.columns if c != label_col]
print("Feature cols count:", len(FEATURE_COLS))

prod_features = prod_df[FEATURE_COLS].copy()
bt_local = Path("w4_production_features_only.csv")
prod_features.to_csv(bt_local, index=False, header=False)

s3_bt_input = sm_session.upload_data(str(bt_local), key_prefix=f"{W4_PREFIX}/batch")

transformer = xgb_est.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket}/{W4_PREFIX}/batch/outputs",
    accept="text/csv",
    assemble_with="Line",
)

transformer.transform(data=s3_bt_input, content_type="text/csv", split_type="Line")
transformer.wait()

batch_output_s3 = transformer.output_path
print("Batch output:", batch_output_s3)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-10-07-22-28-22-853


Feature cols count: 18


INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2025-10-07-22-28-23-498


................................./miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-10-07:22:33:55:INFO] No GPUs detected (normal if no gpus installed)
[2025-10-07:22:33:55:INFO] No GPUs detected (normal if no gpus installed)
[2025-10-07:22:33:55:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package

#### ARTIFACTS + DESIGN-DOC SNIPPET + TRACKER (upload to S3)

In [16]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_cm(cm, title, path):
    plt.figure()
    plt.imshow(cm, interpolation='nearest'); plt.title(title); plt.colorbar()
    plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.tight_layout(); plt.savefig(path); plt.close()

# Confusion matrices (test set)
bm_pred = (LogisticRegression(max_iter=1000).fit(train[bm_feats], y_train)
           .predict_proba(test[bm_feats])[:,1] >= 0.5).astype(int)
bm_cm   = confusion_matrix(y_test, bm_pred)
xgb_cm  = confusion_matrix(y_test, xgb_pred)

art_dir = Path("w4_artifacts"); art_dir.mkdir(exist_ok=True)
plot_cm(bm_cm,  "Baseline CM", art_dir/"baseline_cm.png")
plot_cm(xgb_cm, "XGBoost CM",  art_dir/"xgb_cm.png")
with open(art_dir/"metrics_compare.json","w") as f:
    json.dump(metrics_compare, f, indent=2)

def up(local, key):
    boto3.client("s3").upload_file(str(local), bucket, f"{W4_PREFIX}/{key}")
    return f"s3://{bucket}/{W4_PREFIX}/{key}"

metrics_s3 = up(art_dir/"metrics_compare.json", "metrics_compare.json")
bm_cm_s3   = up(art_dir/"baseline_cm.png",      "baseline_cm.png")
xgb_cm_s3  = up(art_dir/"xgb_cm.png",           "xgb_cm.png")

design_doc_snippet = f"""
### Week 4 Findings — Model Development & Deployment

**Benchmark (LogReg on Age + SystolicBP)**  
Acc: {baseline_metrics['accuracy']:.3f} | Prec: {baseline_metrics['precision']:.3f} | Rec: {baseline_metrics['recall']:.3f} | F1: {baseline_metrics['f1']:.3f} | AUC: {baseline_metrics['roc_auc']:.3f}

**XGBoost (full features)**  
Acc: {metrics_compare['xgboost']['accuracy']:.3f} | Prec: {metrics_compare['xgboost']['precision']:.3f} | Rec: {metrics_compare['xgboost']['recall']:.3f} | F1: {metrics_compare['xgboost']['f1']:.3f} | AUC: {metrics_compare['xgboost']['roc_auc']:.3f}

**Artifacts**  
- Metrics JSON: {metrics_s3}  
- Baseline CM:  {bm_cm_s3}  
- XGBoost CM:   {xgb_cm_s3}  
- XGBoost Model Artifact: {xgb_model_artifact}  
- Batch Transform Output: {batch_output_s3}
"""
print(design_doc_snippet)

# Tracker (JSON + Markdown)
w4_tracker_dir = Path("w4_tracker"); w4_tracker_dir.mkdir(exist_ok=True)
tracker_w4 = {
    "week": "4",
    "run_id": RUN_ID,
    "week3_prefix": f"s3://{bucket}/{WEEK3_PREFIX}",
    "week4_prefix": f"s3://{bucket}/{W4_PREFIX}",
    "benchmark": baseline_metrics,
    "xgboost": metrics_compare["xgboost"],
    "artifacts": {
        "metrics_json": metrics_s3,
        "baseline_cm": bm_cm_s3,
        "xgb_cm": xgb_cm_s3,
        "model_artifact": xgb_model_artifact,
        "batch_output": batch_output_s3
    }
}
with open(w4_tracker_dir/"team_tracker_update_week4.json","w") as f:
    json.dump(tracker_w4, f, indent=2)

md = f"""# Week 4 Tracker – Maternal Health Risk (RUN: {RUN_ID})

**Week-3 prefix:** s3://{bucket}/{WEEK3_PREFIX}  
**Week-4 prefix:** s3://{bucket}/{W4_PREFIX}

## Benchmark (LogReg on Age + SystolicBP)
Acc: {baseline_metrics['accuracy']:.3f} | Prec: {baseline_metrics['precision']:.3f} | Rec: {baseline_metrics['recall']:.3f} | F1: {baseline_metrics['f1']:.3f} | AUC: {baseline_metrics['roc_auc']:.3f}

# XGBoost (full features)
Acc: {metrics_compare['xgboost']['accuracy']:.3f} | Prec: {metrics_compare['xgboost']['precision']:.3f} | Rec: {metrics_compare['xgboost']['recall']:.3f} | F1: {metrics_compare['xgboost']['f1']:.3f} | AUC: {metrics_compare['xgboost']['roc_auc']:.3f}

# Artifacts
- Metrics JSON: {metrics_s3}
- Baseline CM:  {bm_cm_s3}
- XGBoost CM:   {xgb_cm_s3}
- Model:        {xgb_model_artifact}
- Batch Output: {batch_output_s3}
"""
with open(w4_tracker_dir/"team_tracker_update_week4.md","w") as f:
    f.write(md)

boto3.client("s3").upload_file(str(w4_tracker_dir/"team_tracker_update_week4.json"), bucket, f"{W4_PREFIX}/team_tracker_update_week4.json")
boto3.client("s3").upload_file(str(w4_tracker_dir/"team_tracker_update_week4.md"),   bucket, f"{W4_PREFIX}/team_tracker_update_week4.md")

print("Week-4 tracker written & uploaded →", f"s3://{bucket}/{W4_PREFIX}/team_tracker_update_week4.*")


### Week 4 Findings — Model Development & Deployment

**Benchmark (LogReg on Age + SystolicBP)**  
Acc: 0.765 | Prec: 0.719 | Rec: 0.697 | F1: 0.708 | AUC: 0.791

**XGBoost (full features)**  
Acc: 0.988 | Prec: 1.000 | Rec: 0.970 | F1: 0.985 | AUC: 0.999

**Artifacts**  
- Metrics JSON: s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946/metrics_compare.json  
- Baseline CM:  s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946/baseline_cm.png  
- XGBoost CM:   s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946/xgb_cm.png  
- XGBoost Model Artifact: s3://sagemaker-us-east-1-849121223812/sagemaker-xgboost-2025-10-07-22-24-35-764/output/model.tar.gz  
- Batch Transform Output: s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946/batch/outputs

Week-4 tracker written & uploaded → s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946/team_tracker_u

## AAI-540 - Week 5: Monitoring

In [17]:
"""
AAI-540 — Week 5 Monitoring (Maternal Health Risk Prediction)
This notebook cell is a complete, reproducible, commented script that delivers:
  1) Model monitors (Data Quality + Model Quality)
  2) Data monitors (data capture + baseline)
  3) Infrastructure monitors (CloudWatch alarms)
  4) CloudWatch dashboard
  5) S3-hosted Week-5 tracker (MD + JSON) for the Team Project Update

"""
import os, io, json, time
from pathlib import Path

import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker.image_uris import retrieve
from sagemaker.predictor import Predictor
from sagemaker.model_monitor import (
    DataCaptureConfig,
    DefaultModelMonitor,
    ModelQualityMonitor,
    CronExpressionGenerator,
    DatasetFormat,
    EndpointInput,      # older/newer SDK-friendly way to pass MQ attributes
)

# AWS context & run IDs
boto_sess  = boto3.session.Session()
region     = boto_sess.region_name
sm_sess    = Session(boto_sess)
role       = get_execution_role()
s3c        = boto3.client("s3")
cw         = boto3.client("cloudwatch")
bucket     = sm_sess.default_bucket()

RUN_ID    = time.strftime("%Y%m%d-%H%M%S")
W5_PREFIX = f"aai540/maternal-risk/week5/{RUN_ID}"

print(f"[INFO] Region={region}  Bucket={bucket}  Run={RUN_ID}")

# Load Week-4 tracker -> model artifact + Week-3 splits
WEEK4_PREFIX = os.environ.get("WEEK4_PREFIX")
if not WEEK4_PREFIX:
    base = "aai540/maternal-risk/week4/"
    r = s3c.list_objects_v2(Bucket=bucket, Prefix=base, Delimiter="/")
    candidates = [cp["Prefix"].rstrip("/") for cp in r.get("CommonPrefixes", [])]
    assert candidates, f"No Week-4 runs found in s3://{bucket}/{base}"
    WEEK4_PREFIX = sorted(candidates)[-1]

w4_tracker_key = f"{WEEK4_PREFIX}/team_tracker_update_week4.json"
w4 = json.loads(s3c.get_object(Bucket=bucket, Key=w4_tracker_key)["Body"].read().decode("utf-8"))
MODEL_ARTIFACT = w4["artifacts"]["model_artifact"]
WEEK3_PREFIX   = w4["week3_prefix"].replace(f"s3://{bucket}/", "")

print("Using Week-4:", f"s3://{bucket}/{WEEK4_PREFIX}")
print("Week-3:     ", f"s3://{bucket}/{WEEK3_PREFIX}")
print("Model:      ", MODEL_ARTIFACT)

[INFO] Region=us-east-1  Bucket=sagemaker-us-east-1-849121223812  Run=20251007-224036
Using Week-4: s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week4/20251007-221946
Week-3:      s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week3/20251007-221711
Model:       s3://sagemaker-us-east-1-849121223812/sagemaker-xgboost-2025-10-07-22-24-35-764/output/model.tar.gz


#### Deploy endpoint with data capture (we'll send a few warm-up inferences)

In [18]:
xgb_image = retrieve("xgboost", region, version="1.7-1")
ENDPOINT_NAME = f"mhr-xgb-w5-{RUN_ID}"

model = Model(
    image_uri=xgb_image,
    model_data=MODEL_ARTIFACT,
    role=role,
    sagemaker_session=sm_sess,
    name=f"mhr-xgb-model-{RUN_ID}",
)

data_capture_prefix = f"{W5_PREFIX}/datacapture"
data_capture = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,                             # capture all early traffic
    destination_s3_uri=f"s3://{bucket}/{data_capture_prefix}",
    capture_options=["Input", "Output"],
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=ENDPOINT_NAME,
    data_capture_config=data_capture,
)

# Re-attach safeguard (in case some Studio images return None)
if predictor is None:
    predictor = Predictor(endpoint_name=ENDPOINT_NAME, sagemaker_session=sm_sess)

print("Endpoint ready:", ENDPOINT_NAME)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: mhr-xgb-model-20251007-224036
INFO:sagemaker:Creating endpoint-config with name mhr-xgb-w5-20251007-224036
INFO:sagemaker:Creating endpoint with name mhr-xgb-w5-20251007-224036


------!Endpoint ready: mhr-xgb-w5-20251007-224036


#### Warm up endpoint & upload matching ground-truth labels

In [19]:
def read_csv_s3(key: str) -> pd.DataFrame:
    obj = s3c.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(io.BytesIO(obj["Body"].read()))

val = read_csv_s3(f"{WEEK3_PREFIX}/val.csv")
feature_cols = [c for c in val.columns if c != "label"]

# Send 25 rows to create captured traffic
payload = "\n".join(",".join(map(str, row)) for row in val[feature_cols].iloc[:25].values)
_ = predictor.predict(payload, initial_args={"ContentType":"text/csv"})

# Upload the corresponding labels (one per line, no header)
gt_dir = Path("gt"); gt_dir.mkdir(exist_ok=True)
gt_file = gt_dir / "val_labels.csv"
val["label"].iloc[:25].to_csv(gt_file, index=False, header=False)
s3c.upload_file(str(gt_file), bucket, f"{W5_PREFIX}/ground-truth/val_labels.csv")
GROUND_TRUTH_S3 = f"s3://{bucket}/{W5_PREFIX}/ground-truth/val_labels.csv"

#### Data Quality monitor - build baseline & schedule hourly

In [20]:
# Baseline from Week-4 train CSV (drop label column 0 to reflect inference inputs)
resp = s3c.list_objects_v2(Bucket=bucket, Prefix=f"{WEEK4_PREFIX}/xgb/")
train_keys = [o["Key"] for o in resp.get("Contents", []) if o["Key"].endswith("w4_xgb_train.csv")]
assert train_keys, "Week-4 XGB train CSV not found."
xgb_train_key = train_keys[0]

raw = pd.read_csv(io.BytesIO(s3c.get_object(Bucket=bucket, Key=xgb_train_key)["Body"].read()), header=None)
raw.drop(columns=[0]).to_csv("xgb_train_inputs.csv", index=False, header=False)
baseline_inputs_key = f"{W5_PREFIX}/baselines/xgb_train_inputs.csv"
s3c.upload_file("xgb_train_inputs.csv", bucket, baseline_inputs_key)
BASELINE_INPUTS_S3 = f"s3://{bucket}/{baseline_inputs_key}"

dq_output = f"s3://{bucket}/{W5_PREFIX}/monitoring/data-quality"
dqm = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,                 # < 1 hour cadence
    sagemaker_session=sm_sess,
)
dqm.suggest_baseline(
    baseline_dataset=BASELINE_INPUTS_S3,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=dq_output,
    wait=True,
)

DQ_SCHEDULE = f"mhr-dq-sched-{RUN_ID}"
dqm.create_monitoring_schedule(
    monitor_schedule_name=DQ_SCHEDULE,
    endpoint_input=ENDPOINT_NAME,
    output_s3_uri=dq_output,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)
print("Data Quality schedule:", DQ_SCHEDULE)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2025-10-07-22-45-14-608


...............2025-10-07 22:47:48.206741: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-10-07 22:47:48.206778: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-10-07 22:47:49.852620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-10-07 22:47:49.852651: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-10-07 22:47:49.852675: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-100-50.ec2.internal): /proc/driver/nvidia/version does not exist
2025-10-07 22:47:49.852961: I tens

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: mhr-dq-sched-20251007-224036


Data Quality schedule: mhr-dq-sched-20251007-224036


#### Model Quality monitor - schedule hourly on previous full hour

In [21]:
mq_output = f"s3://{bucket}/{W5_PREFIX}/monitoring/model-quality"
mqm = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,                 # < 1 hour cadence
    sagemaker_session=sm_sess,
)

ENDPOINT_DEST = "/opt/ml/processing/input/endpoint"

endpoint_input = EndpointInput(
    endpoint_name=ENDPOINT_NAME,
    destination=ENDPOINT_DEST,
    # previous full hour window (works reliably in us-east-1)
    start_time_offset="-PT2H",
    end_time_offset="-PT1H",
    # model output is a single probability in CSV column 0
    probability_attribute="0",
    probability_threshold_attribute=0.5,
)

MQ_SCHEDULE = f"mhr-mq-sched-{RUN_ID}"
mqm.create_monitoring_schedule(
    monitor_schedule_name=MQ_SCHEDULE,
    endpoint_input=endpoint_input,
    ground_truth_input=GROUND_TRUTH_S3,
    problem_type="BinaryClassification",
    output_s3_uri=mq_output,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)
print("Model Quality schedule:", MQ_SCHEDULE)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: mhr-mq-sched-20251007-224036


Model Quality schedule: mhr-mq-sched-20251007-224036


#### Infrastructure monitors  CloudWatch alarms

In [22]:
ALARM_PREFIX = f"MHR-W5-{RUN_ID}"

def put_alarm(metric, stat, comp, thresh, period=60, evals=1, unit=None):
    dims=[{"Name":"EndpointName","Value":ENDPOINT_NAME},
          {"Name":"VariantName","Value":"AllTraffic"}]
    params=dict(
        AlarmName=f"{ALARM_PREFIX}-{metric}",
        Namespace="AWS/SageMaker",
        MetricName=metric,
        Dimensions=dims,
        Statistic=stat,
        Period=period,
        EvaluationPeriods=evals,
        ComparisonOperator=comp,
        Threshold=thresh,
        ActionsEnabled=False,                # flip True + SNS if we want emails
        TreatMissingData="notBreaching",
    )
    if unit: params["Unit"]=unit
    cw.put_metric_alarm(**params)

put_alarm("ModelLatency",    "Average", "GreaterThanThreshold", 60000)
put_alarm("OverheadLatency", "Average", "GreaterThanThreshold", 60000)
put_alarm("5XXErrors",       "Sum",     "GreaterThanThreshold", 0)
put_alarm("Invocations",     "Sum",     "GreaterThanThreshold", 500)
put_alarm("CPUUtilization",  "Average", "GreaterThanThreshold", 95, unit="Percent")
put_alarm("MemoryUtilization","Average","GreaterThanThreshold", 95, unit="Percent")

#### CloudWatch dashboard

In [23]:
DASHBOARD = f"MHR-W5-Dashboard-{RUN_ID}"
widgets = [
  {"type":"metric","width":12,"height":6,"properties":{
      "region":region,"title":"Latency (ms)",
      "metrics":[["AWS/SageMaker","ModelLatency","EndpointName",ENDPOINT_NAME,"VariantName","AllTraffic"],
                 [".","OverheadLatency",".",".",".","."]],
      "period":60,"stat":"Average","view":"timeSeries"}},
  {"type":"metric","width":12,"height":6,"properties":{
      "region":region,"title":"Invocations & 5XX",
      "metrics":[["AWS/SageMaker","Invocations","EndpointName",ENDPOINT_NAME,"VariantName","AllTraffic"],
                 [".","5XXErrors",".",".",".","."]],
      "period":60,"stat":"Sum","view":"timeSeries"}}
]
cw.put_dashboard(DashboardName=DASHBOARD, DashboardBody=json.dumps({"widgets":widgets}))
print("CloudWatch dashboard:", DASHBOARD)

CloudWatch dashboard: MHR-W5-Dashboard-20251007-224036


#### Week-5 tracker (JSON + MD)

In [24]:
tracker = {
    "week": 5,
    "run_id": RUN_ID,
    "week4_prefix": f"s3://{bucket}/{WEEK4_PREFIX}",
    "week5_prefix": f"s3://{bucket}/{W5_PREFIX}",
    "endpoint": ENDPOINT_NAME,
    "datacapture_s3": f"s3://{bucket}/{data_capture_prefix}",
    "schedules": {"data_quality": DQ_SCHEDULE, "model_quality": MQ_SCHEDULE},
    "outputs": {"data_quality": dq_output, "model_quality": mq_output, "ground_truth": GROUND_TRUTH_S3},
    "cloudwatch": {"alarms_prefix": ALARM_PREFIX, "dashboard": DASHBOARD}
}
Path("w5_tracker").mkdir(exist_ok=True)
json.dump(tracker, open("w5_tracker/team_tracker_update_week5.json","w"), indent=2)
open("w5_tracker/team_tracker_update_week5.md","w").write(
    f"# Week 5 Tracker – Maternal Health Risk (RUN: {RUN_ID})\n\n"
    f"**Week-4:** s3://{bucket}/{WEEK4_PREFIX}\n"
    f"**Week-5:** s3://{bucket}/{W5_PREFIX}\n\n"
    f"## Endpoint\n- {ENDPOINT_NAME}\n- Data Capture: s3://{bucket}/{data_capture_prefix}\n\n"
    f"## Monitoring Schedules\n- Data Quality: {DQ_SCHEDULE}\n- Model Quality: {MQ_SCHEDULE}\n\n"
    f"## Outputs\n- DQ: {dq_output}\n- MQ: {mq_output}\n- GT: {GROUND_TRUTH_S3}\n\n"
    f"## CloudWatch\n- Alarms prefix: {ALARM_PREFIX}\n- Dashboard: {DASHBOARD}\n"
)
s3c.upload_file("w5_tracker/team_tracker_update_week5.json", bucket, f"{W5_PREFIX}/team_tracker_update_week5.json")
s3c.upload_file("w5_tracker/team_tracker_update_week5.md",   bucket, f"{W5_PREFIX}/team_tracker_update_week5.md")

print("Week-5 monitoring setup complete.")

Week-5 monitoring setup complete.


### week5_generate_reports.py <-- We'll run this after at least one schedule execution

In [27]:
"""
AAI-540 — Week 5 Reporting (student version)
- Finds the latest Week-5 run
- Lists Data/Model Quality outputs in S3
- Writes compact MD + JSON report for Team Update
"""

import json, io
from pathlib import Path
import boto3
from sagemaker.session import Session

s3  = boto3.client("s3")
bkt = Session().default_bucket()

# Find latest Week-5 folder
base="aai540/maternal-risk/week5/"
resp=s3.list_objects_v2(Bucket=bkt, Prefix=base, Delimiter="/")
runs=[cp["Prefix"].rstrip("/") for cp in resp.get("CommonPrefixes",[])]
assert runs, f"No Week-5 runs under s3://{bkt}/{base}"
w5=sorted(runs)[-1]

# Load Week-5 tracker
tracker=json.loads(s3.get_object(Bucket=bkt, Key=f"{w5}/team_tracker_update_week5.json")["Body"].read())
dq_out=tracker["outputs"]["data_quality"]; mq_out=tracker["outputs"]["model_quality"]

def list_all(prefix_uri):
    """List all keys under a given s3://bucket/prefix and return as full s3 URIs."""
    prefix=prefix_uri.replace(f"s3://{bkt}/","")
    out=[]; token=None
    while True:
        kw=dict(Bucket=bkt, Prefix=prefix)
        if token: kw["ContinuationToken"]=token
        r=s3.list_objects_v2(**kw)
        out += [o["Key"] for o in r.get("Contents",[])]
        token=r.get("NextContinuationToken")
        if not token: break
    return [f"s3://{bkt}/{k}" for k in out]

dq_files=list_all(dq_out)
mq_files=list_all(mq_out)

report={
  "week":5,
  "run":w5,
  "data_quality":{
    "statistics": next((f for f in dq_files if f.endswith("statistics.json")), "N/A"),
    "constraints": next((f for f in dq_files if f.endswith("constraints.json")), "N/A"),
    "violations": next((f for f in dq_files if "constraint_violations.json" in f), "N/A"),
  },
  "model_quality":{
    "files": [f for f in mq_files if f.endswith((".json",".csv"))][-10:]
  }
}

Path("w5_reports").mkdir(exist_ok=True)
open("w5_reports/monitoring_report.md","w").write(
  "# Week 5 Monitoring Report\n\n"
  f"**Run:** {w5}\n\n"
  "## Data Quality\n"
  f"- Statistics: {report['data_quality']['statistics']}\n"
  f"- Constraints: {report['data_quality']['constraints']}\n"
  f"- Violations: {report['data_quality']['violations']}\n\n"
  "## Model Quality (latest files)\n" + "\n".join(f"- {p}" for p in report["model_quality"]["files"])
)
open("w5_reports/monitoring_report.json","w").write(json.dumps(report, indent=2))

s3.upload_file("w5_reports/monitoring_report.md",   bkt, f"{w5}/monitoring_report.md")
s3.upload_file("w5_reports/monitoring_report.json", bkt, f"{w5}/monitoring_report.json")

print("Report written -->", f"s3://{bkt}/{w5}/monitoring_report.md")

Report written --> s3://sagemaker-us-east-1-849121223812/aai540/maternal-risk/week5/20251007-224036/monitoring_report.md


### Week 5 cleanup 

In [31]:
"""
AAI-540 — Week 5 Cleanup
Deletes: monitoring schedules, endpoint (+config), CW dashboard & alarms.
Artifacts remain in S3 for grading.


import json, boto3
from sagemaker.session import Session

s3 = boto3.client("s3"); sm = boto3.client("sagemaker"); cw=boto3.client("cloudwatch")
bkt=Session().default_bucket()

# latest Week-5 run
base="aai540/maternal-risk/week5/"
resp=s3.list_objects_v2(Bucket=bkt, Prefix=base, Delimiter="/")
w5=sorted([cp["Prefix"].rstrip("/") for cp in resp.get("CommonPrefixes",[])])[-1]

trk=json.loads(s3.get_object(Bucket=bkt, Key=f"{w5}/team_tracker_update_week5.json")["Body"].read())
ep=trk["endpoint"]; dq=trk["schedules"]["data_quality"]; mq=trk["schedules"]["model_quality"]
dash=trk["cloudwatch"]["dashboard"]; prefix=trk["cloudwatch"]["alarms_prefix"]

# stop & delete schedules
for name in [dq, mq]:
    try: sm.stop_monitoring_schedule(MonitoringScheduleName=name)
    except: pass
    try: sm.delete_monitoring_schedule(MonitoringScheduleName=name)
    except: pass

# delete endpoint (+ config)
try: sm.delete_endpoint(EndpointName=ep)
except: pass
try: sm.delete_endpoint_config(EndpointConfigName=ep)
except: pass

# delete dashboard
try: cw.delete_dashboards(DashboardNames=[dash])
except: pass

# delete alarms with our prefix
alarms=cw.describe_alarms(AlarmNamePrefix=prefix).get("MetricAlarms",[])
if alarms:
    cw.delete_alarms(AlarmNames=[a["AlarmName"] for a in alarms])

print("🧹 Cleanup complete for run:", w5)  
"""

'\nAAI-540 — Week 5 Cleanup\nDeletes: monitoring schedules, endpoint (+config), CW dashboard & alarms.\nArtifacts remain in S3 for grading.\n\n\nimport json, boto3\nfrom sagemaker.session import Session\n\ns3 = boto3.client("s3"); sm = boto3.client("sagemaker"); cw=boto3.client("cloudwatch")\nbkt=Session().default_bucket()\n\n# latest Week-5 run\nbase="aai540/maternal-risk/week5/"\nresp=s3.list_objects_v2(Bucket=bkt, Prefix=base, Delimiter="/")\nw5=sorted([cp["Prefix"].rstrip("/") for cp in resp.get("CommonPrefixes",[])])[-1]\n\ntrk=json.loads(s3.get_object(Bucket=bkt, Key=f"{w5}/team_tracker_update_week5.json")["Body"].read())\nep=trk["endpoint"]; dq=trk["schedules"]["data_quality"]; mq=trk["schedules"]["model_quality"]\ndash=trk["cloudwatch"]["dashboard"]; prefix=trk["cloudwatch"]["alarms_prefix"]\n\n# stop & delete schedules\nfor name in [dq, mq]:\n    try: sm.stop_monitoring_schedule(MonitoringScheduleName=name)\n    except: pass\n    try: sm.delete_monitoring_schedule(MonitoringSc

## Week 6: Implement CI/CD Pipeline to automate training, evaluation, and deployment.

#### Setup + dataset resolver (auto-detect the latest CSV on S3)

In [104]:
# 1) Resolves latest Week-3 splits from S3 (train/val/test CSV URIs).
# 2) Processing V4 (robust) normalizes schemas -> writes "train.csv" for each split.
# 3) Trains XGBoost on train, validates on val.
# 4) Evaluates the trained model on the test split inside an XGBoost ScriptProcessor.
# 5) Reads AUC from evaluation.json; quality gate passes if AUC >= threshold.
# 6) If pass, registers model into Model Package Group "MaternalHealthRisk".
#
# Key debugging upgrades:
# - explicit LabRole used everywhere (jobs + pipeline resource).
# - uses Join() for S3 paths (no string concat on pipeline variables).
# - evaluation step runs in XGBoost image (so xgboost is importable).
# - processing/evaluate scripts print helpful context to CloudWatch.
# - caching disabled + V4 step names to force upload of fresh code.

import os, time, boto3
from pathlib import Path
from sagemaker.session import Session
from sagemaker.image_uris import retrieve
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.parameters import (
    ParameterString, ParameterFloat, ParameterInteger
)
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.functions import JsonGet, Join
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics

# environment
boto_sess  = boto3.session.Session()
sm_sess    = Session(boto_sess)
region     = boto_sess.region_name
bucket     = sm_sess.default_bucket()
ROLE_ARN   = "arn:aws:iam::849121223812:role/LabRole"

RUN_ID        = time.strftime("%Y%m%d-%H%M%S")
PIPELINE_NAME = "MaternalHealthRisk-CICD"
BASE_PREFIX   = f"aai540/maternal-risk/week6/{RUN_ID}"

Path("week6_code").mkdir(parents=True, exist_ok=True)
assert Path("week6_code/processing_v4.py").exists()
assert Path("week6_code/evaluate.py").exists()

# dataset resolver
def resolve_week3_csv():
    s3 = boto3.client("s3")
    base = "aai540/maternal-risk/week3/"
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=base, Delimiter="/")
    runs = sorted(cp["Prefix"].rstrip("/") for cp in resp.get("CommonPrefixes", []))
    assert runs, "No Week-3 runs found. Please run Week 3 prep first."
    latest = runs[-1]
    return (f"s3://{bucket}/{latest}/train.csv",
            f"s3://{bucket}/{latest}/val.csv",
            f"s3://{bucket}/{latest}/test.csv")

TRAIN_S3, VAL_S3, TEST_S3 = resolve_week3_csv()

# pipeline parameters (adds instance types/counts)
p_base_prefix   = ParameterString("BasePrefix", default_value=BASE_PREFIX)
p_auc_gate      = ParameterFloat("AUCThreshold", default_value=0.90)
p_max_depth     = ParameterInteger("MaxDepth", default_value=5)
p_eta           = ParameterFloat("Eta", default_value=0.2)
p_rounds        = ParameterInteger("NumRounds", default_value=200)

# define the parameters that the error complained about (and wire them)
p_proc_instance_type  = ParameterString("ProcInstanceType",  default_value="ml.m5.large")
p_eval_instance_type  = ParameterString("EvalInstanceType",  default_value="ml.m5.large")
p_train_instance_type = ParameterString("TrainInstanceType", default_value="ml.m5.large")
p_train_instance_cnt  = ParameterInteger("TrainInstanceCount", default_value=1)

# images
sk_image  = retrieve("sklearn", region, version="1.2-1")
xgb_image = retrieve("xgboost", region, version="1.7-1")

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


#### PROCESS V4 (no cache; V4 names; robust)

In [106]:
# PROCESS (v4)
proc = ScriptProcessor(
    image_uri=sk_image,
    role=ROLE_ARN,
    instance_type=p_proc_instance_type,   # wired to parameter
    instance_count=1,
    command=["python3"],
    sagemaker_session=sm_sess
)

def proc_step(name, s3_uri, split_name):
    return ProcessingStep(
        name=name,
        processor=proc,
        code="week6_code/processing_v4.py",
        inputs=[ProcessingInput(source=s3_uri, destination="/opt/ml/processing/input")],
        outputs=[ProcessingOutput(
            output_name="out",
            source="/opt/ml/processing/output",
            destination=f"s3://{bucket}/{BASE_PREFIX}/data/{split_name}"
        )]
    )

step_proc_train = proc_step("ProcessTrainV4", TRAIN_S3, "train")
step_proc_val   = proc_step("ProcessValV4",   VAL_S3,   "val")
step_proc_test  = proc_step("ProcessTestV4",  TEST_S3,  "test")

#### Train

In [107]:
est = Estimator(
    image_uri=xgb_image,
    role=ROLE_ARN,
    instance_count=p_train_instance_cnt,      
    instance_type=p_train_instance_type,      
    sagemaker_session=sm_sess,
    hyperparameters={
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "max_depth": p_max_depth,
        "eta": p_eta,
        "num_round": p_rounds,
        "subsample": 0.8,
        "colsample_bytree": 0.8
    }
)

train_file_uri = Join(on="/", values=[
    step_proc_train.properties.ProcessingOutputConfig.Outputs["out"].S3Output.S3Uri, "train.csv"
])
val_file_uri = Join(on="/", values=[
    step_proc_val.properties.ProcessingOutputConfig.Outputs["out"].S3Output.S3Uri, "train.csv"
])

step_train = TrainingStep(
    name="TrainXGB",
    estimator=est,
    inputs={
        "train": TrainingInput(s3_data=train_file_uri, content_type="text/csv"),
        "validation": TrainingInput(s3_data=val_file_uri, content_type="text/csv")
    }
)

#### EVALUATE (runs in XGBoost image; looks for train.csv first)

In [111]:
eval_proc = ScriptProcessor(
    image_uri=xgb_image,
    role=ROLE_ARN,
    instance_type=p_eval_instance_type,   # ← wired to parameter
    instance_count=1,
    command=["python3"],
    sagemaker_session=sm_sess
)

test_file_uri = Join(on="/", values=[
    step_proc_test.properties.ProcessingOutputConfig.Outputs["out"].S3Output.S3Uri, "train.csv"
])

eval_out_s3  = f"s3://{bucket}/{BASE_PREFIX}/evaluation"
eval_report  = PropertyFile(name="EvalReport", output_name="eval", path="evaluation.json")

step_eval = ProcessingStep(
    name="EvaluateOnTestV2",
    processor=eval_proc,
    code="week6_code/evaluate.py",
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=test_file_uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[ProcessingOutput(
        output_name="eval",
        source="/opt/ml/processing/evaluation",
        destination=eval_out_s3
    )],
    property_files=[eval_report]
)

#### QUALITY GATE + REGISTER

In [112]:
cond_ok = ConditionGreaterThanOrEqualTo(
    left=JsonGet(step_name=step_eval.name, property_file=eval_report,
                 json_path="binary_classification_metrics.roc_auc"),
    right=p_auc_gate
)

metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="/", values=[eval_out_s3, "evaluation.json"]),
        content_type="application/json"
    )
)

register = RegisterModel(
    name="RegisterBestXGB",
    estimator=est,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="MaternalHealthRisk",
    model_metrics=metrics
)

step_gate = ConditionStep(name="QualityGate", conditions=[cond_ok], if_steps=[register], else_steps=[])

#### assemble, upsert, start

In [113]:
pipeline = Pipeline(
    name=PIPELINE_NAME,
    parameters=[
        p_base_prefix, p_auc_gate, p_max_depth, p_eta, p_rounds,
        p_proc_instance_type, p_eval_instance_type,
        p_train_instance_type, p_train_instance_cnt
    ],
    steps=[step_proc_train, step_proc_val, step_proc_test, step_train, step_eval, step_gate],
    sagemaker_session=sm_sess
)

print("Compiling pipeline definition…")
_ = pipeline.definition()

print("Upserting pipeline with LabRole…")
pipeline.upsert(role_arn=ROLE_ARN)

print("Starting execution…")
execution = pipeline.start(
    parameters={
        "BasePrefix": BASE_PREFIX,
        "AUCThreshold": 0.90,
        "MaxDepth": 5,
        "Eta": 0.2,
        "NumRounds": 200,
        "ProcInstanceType":  "ml.m5.large",
        "EvalInstanceType":  "ml.m5.large",
        "TrainInstanceType": "ml.m5.large",
        "TrainInstanceCount": 1
    },
    execution_display_name=f"run-{RUN_ID}"
)
print("Execution ARN:", execution.arn)

Compiling pipeline definition…


Upserting pipeline with LabRole…


Starting execution…
Execution ARN: arn:aws:sagemaker:us-east-1:849121223812:pipeline/MaternalHealthRisk-CICD/execution/lniyz00otftm
